In [1031]:
!pip install --quiet python-dotenv
!pip install --quiet pandas
!pip install --quiet psycopg2

In [1]:
import os
import psycopg2
import pandas as pd
import numpy as np
from dotenv import load_dotenv
from decimal import Decimal, getcontext

In [2]:
In [25]: pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [3]:
# Load environment variables from .env file
load_dotenv()

True

In [4]:
configid = -77
datasetid = -770
# runid = 19  # lead_time not taken into account
runid = 20
period = 0, 1, 2
sorting = 'DESC'

In [5]:
# Get the database connection parameters from environment variables
db_params = {
    'host': os.getenv('DB_HOST'),
    'port': os.getenv('DB_PORT'),
    'database': os.getenv('DB_NAME'),
    'user': os.getenv('DB_USERNAME'),
    'password': os.getenv('DB_PASSWORD'),
}

In [6]:
# Connect to the database
conn = psycopg2.connect(**db_params)
cursor = conn.cursor()

# Optimizer Production Lead time and capacity

In [15]:
# Optimizer Production
# Query the products from the optimizer_production table
cursor.execute(f"""
    SELECT *
    FROM optimizer_production
    WHERE datasetid = {datasetid} AND period IN {period}
""")
production_rows = cursor.fetchall()

In [17]:
# Save the results in a DataFrame
df_production = pd.DataFrame(production_rows, columns=[desc[0] for desc in cursor.description])

# Cast integer datatypes
df_production['period'] = df_production['period'].astype(int)
df_production['duration'] = df_production['duration'].astype(int)

# Drop unnecessary columns
production_cols = ['location', 'product', 'bomnum', 'resource', 'capacity', 'coefficient', 'period', 'duration']
df_production = df_production[production_cols].copy()

# Drop duplicates
df_production = df_production.drop_duplicates()

In [18]:
df_production

,location,product,bomnum,resource,capacity,coefficient,period,duration
0,obninsk,steel,steelobninsk1,steel_converter_1,10000,1,0,1
1,obninsk,rebar,rebarobninsk1,rebar_machine_1,5000,1,0,1
2,obninsk,reinforcedrebar,reinforcedrebarobninsk1,reinforced_rebar_machine_1,5000,1,0,1
3,obninsk,rebar,rebarobninsk1,cutting machine_1,5000,1,0,1
4,obninsk,reinforcedrebar,reinforcedrebarobninsk1,cutting machine_1,5000,1,0,1
5,obninsk,rebar,rebarobninsk1,heater_1,10000,1,0,1
6,obninsk,steel,steelobninsk1,heater_1,10000,1,0,1
7,lipetsk,steel,steellipetsk1,steel_converter_2,10000,1,0,1
8,lipetsk,rebar,rebarlipetsk1,rebar_machine_2,5000,1,0,1
9,lipetsk,reinforcedrebar,reinforcedrebarlipetsk1,reinforced_rebar_machine_2,5000,1,0,1


# Results Production

In [31]:
# Results Production
# Query the products from the results_production table
cursor.execute(f"""
    SELECT *
    FROM results_production
    WHERE configid = {configid} AND runid = {runid} AND period IN {period}
    ORDER BY CAST(period AS int) {sorting}
""")
results_production_rows = cursor.fetchall()

In [32]:
# Save the results in a DataFrame
df_results_production = pd.DataFrame(results_production_rows, columns=[desc[0] for desc in cursor.description])
                                                                      
# Filter out numbers close to zero
df_results_production = df_results_production[abs(df_results_production['solutionvalue']) > Decimal('0.1')]
                     
# Drop unnecessary columns
results_production_cols = ['location', 'product', 'bomnum', 'period', 'solutionvalue']
df_results_production = df_results_production[results_production_cols].copy()

# Drop duplicates
df_results_production = df_results_production.drop_duplicates()

In [33]:
df_results_production

,location,product,bomnum,period,solutionvalue
7,obninsk,rebar,rebarobninsk1,1,333.3333333333333
8,obninsk,reinforcedrebar,reinforcedrebarobninsk1,1,166.66666666666666
10,lipetsk,rebar,rebarlipetsk1,1,235.2941176470588
11,lipetsk,reinforcedrebar,reinforcedrebarlipetsk1,1,112.0448179271709
15,obninsk,steel,steelobninsk1,1,416.66666641666666
21,rome,beer,beerrome1,0,499.9999999
22,lipetsk,reinforcedrebar,reinforcedrebarlipetsk1,0,140.05602240896357
23,obninsk,rebar,rebarobninsk1,0,333.3333333333333
24,obninsk,reinforcedrebar,reinforcedrebarobninsk1,0,166.66666666666666
25,piter,car,carpiter1,0,1.9999997999999999


# Merge Production with Lead Time

In [34]:
df_production_leadtime = df_production[['location', 'product', 'bomnum', 'period', 'duration']].copy()

In [35]:
# Merge result production with lead times
df_results_production = pd.merge(df_results_production, df_production_leadtime, on=['location', 'bomnum', 'product', 'period'], how='left')

In [36]:
df_results_production = df_results_production.rename(columns={'duration': 'leadtime'})

In [37]:
df_results_production

,location,product,bomnum,period,solutionvalue,leadtime
0,obninsk,rebar,rebarobninsk1,1,333.3333333333333,1
1,obninsk,rebar,rebarobninsk1,1,333.3333333333333,1
2,obninsk,rebar,rebarobninsk1,1,333.3333333333333,1
3,obninsk,reinforcedrebar,reinforcedrebarobninsk1,1,166.66666666666666,1
4,obninsk,reinforcedrebar,reinforcedrebarobninsk1,1,166.66666666666666,1
5,lipetsk,rebar,rebarlipetsk1,1,235.2941176470588,1
6,lipetsk,rebar,rebarlipetsk1,1,235.2941176470588,1
7,lipetsk,rebar,rebarlipetsk1,1,235.2941176470588,1
8,lipetsk,reinforcedrebar,reinforcedrebarlipetsk1,1,112.0448179271709,1
9,lipetsk,reinforcedrebar,reinforcedrebarlipetsk1,1,112.0448179271709,1


In [38]:
# Get capacity dataset
df_capacity = df_production[['location', 'product', 'bomnum', 'resource',
                                        'capacity', 'coefficient', 'period']].copy()

In [41]:
df_production_capacity.sort_values('bomnum', ascending=False)

,location,product,bomnum,resource,capacity,coefficient,period
47,moscow,truck,truckmoscow1,factory_7,10000,1,0
48,moscow,truck,truckmoscow1,factory_7,10000,1,1
49,tushino,train,traintushino1,factory_91,10000,1,0
50,khimki,superjet,superjetkhimki1,airport_1,10000,1,2
20,obninsk,steel,steelobninsk1,heater_1,10000,1,1
28,obninsk,steel,steelobninsk1,steel_converter_1,10000,1,2
14,obninsk,steel,steelobninsk1,steel_converter_1,10000,1,1
0,obninsk,steel,steelobninsk1,steel_converter_1,10000,1,0
6,obninsk,steel,steelobninsk1,heater_1,10000,1,0
34,obninsk,steel,steelobninsk1,heater_1,10000,1,2


# Optimizer Movement Lead time

In [109]:
# Optimizer transportation
# Query the products from the optimizer_transportation table
cursor.execute(f"""
    SELECT *
    FROM optimizer_transportation
    WHERE datasetid = {datasetid} AND period IN {period}
""")
movement_lead_time_rows = cursor.fetchall()

In [110]:
# Save the results in a DataFrame
df_movement_lead_time = pd.DataFrame(movement_lead_time_rows, columns=[desc[0] for desc in cursor.description])

# Cast integers
df_movement_lead_time['duration'] = df_movement_lead_time['duration'].astype(int)

# Drop unnecessary columns
movement_lead_time_cols = ['loc_from', 'loc_to', 'product', 'period', 'transport_type', 'duration']
df_movement_lead_time = df_movement_lead_time[movement_lead_time_cols].copy()

# Drop duplicates
df_movement_lead_time = df_movement_lead_time.drop_duplicates()

In [111]:
df_movement_lead_time

,loc_from,loc_to,product,period,transport_type,duration
0,moscow,kemerovo,steel,0,zhd,0
1,moscow,kemerovo,lime,0,zhd,0
2,moscow,kemerovo,coke,0,zhd,0
3,moscow,kemerovo,rebar,0,zhd,0
4,moscow,kemerovo,reinforcedrebar,0,zhd,0
...,...,...,...,...,...,...
914,kemerovo,obninsk,steel,0,zhd,0
915,kemerovo,obninsk,lime,0,zhd,0
916,kemerovo,obninsk,coke,0,zhd,0
917,kemerovo,obninsk,rebar,0,zhd,0


# Results Movements

In [112]:
# Results Movements
# Query the products from the results_movement table
cursor.execute(f"""
    SELECT *
    FROM results_movement
    WHERE configid = {configid} AND runid = {runid} AND period IN {period}
    ORDER BY CAST(period AS int) {sorting}
""")
results_movement_rows = cursor.fetchall()

In [113]:
# Save the results in a DataFrame
df_results_movement = pd.DataFrame(results_movement_rows, columns=[desc[0] for desc in cursor.description])
# Filter out numbers close to zero
df_results_movement = df_results_movement[abs(df_results_movement['solutionvalue']) > Decimal('0.1')]
# Drop unnecessary columns
df_results_movement_cols = ['loc_from', 'loc_to', 'product', 'period', 'solutionvalue', 'transport_type']
df_results_movement = df_results_movement[df_results_movement_cols].copy()
# Drop duplicates
df_results_movement = df_results_movement.drop_duplicates()

In [114]:
df_results_movement

,loc_from,loc_to,product,period,solutionvalue,transport_type
64,kemerovo,obninsk,steel,2,250.0,auto
83,obninsk,kemerovo,reinforcedrebar,2,166.66666676666665,auto
166,obninsk,rostov-na-donu,rebar,2,250.0,auto
217,lipetsk,omsk,reinforcedrebar,2,249.9999998,auto
224,lipetsk,obninsk,rebar,2,112.04481802717082,auto
227,lipetsk,obninsk,steel,2,137.95518197282922,auto
252,obninsk,rostov-na-donu,rebar,2,20.37815166050413,zhd
408,rome,novgorod,beer,1,250.0,zhd
409,piter,samara,car,1,1.9999999,auto
411,moscow,tosno,truck,1,2.9999999,auto


# Merge Movement with Lead Time

In [ ]:
# Merge result movement with lead times
df_results_movement = pd.merge(df_results_movement, df_movement_lead_time, on=['loc_from', 'loc_to', 'product', 'period', 'transport_type'], how='left')

In [116]:
df_results_movement = df_results_movement.rename(columns={'duration': 'leadtime'})

In [117]:
df_results_movement

,loc_from,loc_to,product,period,solutionvalue,transport_type,leadtime
0,kemerovo,obninsk,steel,2,250.0,auto,0
1,obninsk,kemerovo,reinforcedrebar,2,166.66666676666665,auto,0
2,obninsk,rostov-na-donu,rebar,2,250.0,auto,0
3,lipetsk,omsk,reinforcedrebar,2,249.9999998,auto,0
4,lipetsk,obninsk,rebar,2,112.04481802717082,auto,0
5,lipetsk,obninsk,steel,2,137.95518197282922,auto,0
6,obninsk,rostov-na-donu,rebar,2,20.37815166050413,zhd,0
7,rome,novgorod,beer,1,250.0,zhd,0
8,piter,samara,car,1,1.9999999,auto,0
9,moscow,tosno,truck,1,2.9999999,auto,0


# Results Procurement

In [118]:
# Results Procurement
# Query the products from the results_procurement table
cursor.execute(f"""
    SELECT *
    FROM results_procurement
    WHERE configid = {configid} AND runid = {runid} AND period IN {period}
    ORDER BY CAST(period AS int) {sorting}
""")
results_procurement_rows = cursor.fetchall()

In [119]:
# Save the results in a DataFrame
df_results_procurement = pd.DataFrame(results_procurement_rows, columns=[desc[0] for desc in cursor.description])
# Filter out values close to zero
df_results_procurement = df_results_procurement[abs(df_results_procurement['solutionvalue']) > Decimal('0.1')]
# Drop unnecessary columns
df_results_procurement_cols = ['location', 'product', 'period', 'solutionvalue', 'supplier']
df_results_procurement = df_results_procurement[df_results_procurement_cols].copy()
# Drop duplicates
df_results_procurement = df_results_procurement.drop_duplicates()

In [120]:
df_results_procurement

,location,product,period,solutionvalue,supplier
2,moscow,body,1,2.9999997000000005,s2
3,moscow,chassis,1,3.1499996900000005,s1
4,piter,input,1,39.999995899999995,s


# Initial Stock

In [26]:
# Execute the query to retrieve demands
cursor.execute(f"""
       SELECT *
       FROM optimizer_storage
       WHERE datasetid = {datasetid} AND period IN {period}
       ORDER BY CAST(period AS int) {sorting}
   """)
initial_stock_rows = cursor.fetchall()

In [27]:
# Save the results in a DataFrame
df_initial_stock = pd.DataFrame(initial_stock_rows, columns=[desc[0] for desc in cursor.description])
# Filter out values close to zero
df_initial_stock = df_initial_stock[~df_initial_stock['initialstock'].isna()]
df_initial_stock = df_initial_stock[abs(df_initial_stock['initialstock']) > Decimal('0.1')]
# Drop unnecessary columns
initial_stock_cols = ['location', 'product', 'initialstock', 'period']
df_initial_stock = df_initial_stock[initial_stock_cols].copy()
# Drop duplicates
df_initial_stock = df_initial_stock.drop_duplicates()

In [28]:
df_initial_stock

,location,product,initialstock,period
60,kemerovo,steel,1000.0,0
61,kemerovo,rebar,100.0,0
63,kemerovo,coke,500.0,0
64,kemerovo,lime,500.0,0
65,rostov-na-donu,steel,400.0,0
66,rostov-na-donu,rebar,100.0,0
68,rostov-na-donu,coke,200.0,0
69,rostov-na-donu,lime,200.0,0
70,moscow,lime,200.0,0
71,moscow,coke,200.0,0


# Results Stock

In [192]:
# Results Stock
# Query the products from the results_production table
cursor.execute(f"""
    SELECT *
    FROM results_stock
    WHERE configid = {configid} AND runid = {runid} AND period IN {period}
    ORDER BY CAST(period AS int) {sorting}
""")
results_stock_rows = cursor.fetchall()

In [193]:
# Save the results in a DataFrame
df_results_stock = pd.DataFrame(results_stock_rows, columns=[desc[0] for desc in cursor.description])
# Drop unnecessary columns
df_results_stock_cols = ['location', 'product', 'period', 'solutionvalue']
df_results_stock = df_results_stock[df_results_stock_cols].copy()
# Drop duplicates
df_results_stock = df_results_stock.drop_duplicates()

In [194]:
df_results_stock

,location,product,period,solutionvalue
0,moscow,coke,2,199.99999970000005
1,moscow,lime,2,199.99999970000002
2,lipetsk,steel,2,148.0392139362738
3,omsk,lime,2,0.0
4,omsk,coke,2,199.99999970000005
...,...,...,...,...
85,moscow,reinforcedrebar,0,0.0
86,moscow,rebar,0,0.0
87,moscow,steel,0,1E-7
88,obninsk,lime,0,499.9999998999999


# Merge Result Stock with Initial Stock

In [195]:
# Merge result stock with initial stock
df_results_stock = pd.merge(df_results_stock, df_initial_stock, on=['location', 'product', 'period'], how='left').fillna(0)

In [196]:
# Sort the dataframe by location, product, and period in descending order
df_results_stock = df_results_stock.sort_values(['location', 'product', 'period'], ascending=[True, True, False])

In [197]:
# Create the 'period_spent' column
# Period spent is the difference between next period solutionvalue and current solutionvalue + difference between initial stock and solutionvalue to period spent
df_results_stock['period_spent'] = df_results_stock.groupby(['location', 'product'])['solutionvalue'].diff().shift(-1)

In [198]:
df_results_stock.loc[df_results_stock['period'] == 0, 'period_spent'] = df_results_stock['initialstock'] - df_results_stock['solutionvalue']

In [ ]:
# Create the 'extra_res' column
df_results_stock['extra_res'] = -np.minimum(0, df_results_stock['period_spent'])

# Remove negative 'period_spent'
df_results_stock['period_spent'] = np.maximum(0, df_results_stock['period_spent'])

In [201]:
# Remove rows with zero total stock
# df_results_stock = df_results_stock[(df_results_stock_new['solutionvalue'] + df_results_stock['initialstock'] + abs(df_results_stock['period_spent'])) > Decimal('0.1')]

In [202]:
df_results_stock.head(51)

,location,product,period,solutionvalue,initialstock,period_spent,extra_res
17,kemerovo,coke,2,499.9999997,0,9.999996E-8,0
53,kemerovo,coke,1,499.99999979999996,0,1.0000004E-7,0
63,kemerovo,coke,0,499.9999999,500.0,1E-7,0
16,kemerovo,lime,2,499.99999969999993,0,1.0000003E-7,0
54,kemerovo,lime,1,499.99999979999996,0,1.0000004E-7,0
64,kemerovo,lime,0,499.9999999,500.0,1E-7,0
19,kemerovo,rebar,2,0.0,0,0,0
51,kemerovo,rebar,1,0.0,0,0,0
76,kemerovo,rebar,0,0.0,100.0,100.0,0
18,kemerovo,reinforcedrebar,2,0.0,0,0,0


In [177]:
df_results_stock[(df_results_stock['location'] == 'lipetsk') & (df_results_stock['product'].isin(['lime']))]

,location,product,period,solutionvalue,initialstock,period_spent
13,lipetsk,lime,2,573.9495792319328,0,56.0224090635855
57,lipetsk,lime,1,629.9719882955183,0,70.0280113044817
76,lipetsk,lime,0,699.9999996,500.0,-199.9999996


# Optimizer Demand

In [69]:
# Execute the query to retrieve demands
cursor.execute(f"""
    SELECT *
    FROM optimizer_demand
    WHERE datasetid = {datasetid} AND period IN {period}
""")
demand_rows = cursor.fetchall()

In [159]:
# Save the results in a DataFrame
df_demand = pd.DataFrame(demand_rows, columns=[desc[0] for desc in cursor.description])
# Filter out values close to zero
df_demand = df_demand[abs(df_demand['quantity']) > Decimal('0.1')]
# Drop unnecessary columns
demand_cols = ['location', 'product', 'client', 'quantity', 'price', 'period']
df_demand = df_demand[demand_cols].copy()
# Drop duplicates
df_demand = df_demand.drop_duplicates()

In [160]:
df_demand

,location,product,client,quantity,price,period
0,moscow,rebar,gazprom,500,100,0
1,rostov-na-donu,rebar,mostroi,500,100,0
2,obninsk,steel,mostroi,1000,50,0
3,omsk,reinforcedrebar,mostroi,250,150,0
4,kemerovo,reinforcedrebar,gazprom,250,150,0
5,moscow,rebar,gazprom,500,100,1
6,rostov-na-donu,rebar,mostroi,500,100,1
7,obninsk,steel,mostroi,1000,50,1
8,omsk,reinforcedrebar,mostroi,250,150,1
9,kemerovo,reinforcedrebar,gazprom,250,150,1


# Results Sales

In [161]:
# Results Sales
# Execute the query to retrieve sales
cursor.execute(f"""
    SELECT *
    FROM results_sale
    WHERE configid = {configid} AND runid = {runid} AND period IN {period}
    ORDER BY CAST(period AS int) {sorting}
""")
sale_rows = cursor.fetchall()

In [162]:
# Save the results in a DataFrame
df_results_sale = pd.DataFrame(sale_rows, columns=[desc[0] for desc in cursor.description])
# Filter out values close to zero
df_results_sale = df_results_sale[abs(df_results_sale['solutionvalue']) > Decimal('0.1')]
# Drop unnecessary columns
results_sale_cols = ['location', 'product', 'client', 'solutionvalue', 'period']
df_results_sale = df_results_sale[results_sale_cols].copy()
# Drop duplicates
df_results_sale = df_results_sale.drop_duplicates()

In [163]:
df_results_sale

,location,product,client,solutionvalue,period
2,rostov-na-donu,rebar,mostroi,270.3781517605041,2
3,obninsk,steel,mostroi,1000.0,2
4,omsk,reinforcedrebar,mostroi,250.0,2
5,kemerovo,reinforcedrebar,gazprom,166.66666696666664,2
6,rostov-na-donu,rebar,mostroi,158.33333373333332,1
7,obninsk,steel,mostroi,1000.0,1
8,omsk,reinforcedrebar,mostroi,166.66666696666664,1
9,kemerovo,reinforcedrebar,gazprom,2.1008409361344436,1
10,tosno,truck,avtotech,3.0,1
12,novgorod,beer,lenta,500.0000001,1


In [164]:
# Merge sales with demand
df_results_sale = pd.merge(df_results_sale, df_demand, on=['location', 'product', 'client', 'period'])

In [165]:
df_results_sale

,location,product,client,solutionvalue,period,quantity,price
0,rostov-na-donu,rebar,mostroi,270.3781517605041,2,500,100
1,obninsk,steel,mostroi,1000.0,2,1000,50
2,omsk,reinforcedrebar,mostroi,250.0,2,250,150
3,kemerovo,reinforcedrebar,gazprom,166.66666696666664,2,250,150
4,rostov-na-donu,rebar,mostroi,158.33333373333332,1,500,100
5,obninsk,steel,mostroi,1000.0,1,1000,50
6,omsk,reinforcedrebar,mostroi,166.66666696666664,1,250,150
7,kemerovo,reinforcedrebar,gazprom,2.1008409361344436,1,250,150
8,tosno,truck,avtotech,3.0,1,3,75000
9,novgorod,beer,lenta,500.0000001,1,6500,30


In [166]:
# Calculate the product of solution_value and price
df_results_sale['total_price'] = df_results_sale['solutionvalue'] * df_results_sale['price']

In [170]:
df_results_sale = df_results_sale.sort_values(['period', 'total_price'], ascending=[False, False]).reset_index(drop=True)

In [171]:
df_results_sale

,location,product,client,solutionvalue,period,quantity,price,total_price
0,obninsk,steel,mostroi,1000.0,2,1000,50,50000.0
1,omsk,reinforcedrebar,mostroi,250.0,2,250,150,37500.0
2,rostov-na-donu,rebar,mostroi,270.3781517605041,2,500,100,27037.8151760504100
3,kemerovo,reinforcedrebar,gazprom,166.66666696666664,2,250,150,25000.00004499999600
4,tosno,truck,avtotech,3.0,1,3,75000,225000.0
5,samara,car,avilon,2.0,1,2,40000,80000.0
6,obninsk,steel,mostroi,1000.0,1,1000,50,50000.0
7,omsk,reinforcedrebar,mostroi,166.66666696666664,1,250,150,25000.00004499999600
8,rostov-na-donu,rebar,mostroi,158.33333373333332,1,500,100,15833.33337333333200
9,novgorod,beer,lenta,500.0000001,1,6500,30,15000.0000030


# BOM

In [172]:
# Execute the query to retrieve boms
cursor.execute(f"""
    SELECT *
    FROM optimizer_bom
    WHERE datasetid = {datasetid} AND period IN {period}
""")
bom_rows = cursor.fetchall()

In [173]:
# Save the results in a DataFrame
df_bom = pd.DataFrame(bom_rows, columns=[desc[0] for desc in cursor.description])
# Drop unnecessary columns
df_bom_cols = ['bomnum', 'location', 'product', 'input_output', 'period']
df_bom = df_bom[df_bom_cols].copy()
# Drop duplicates
df_bom = df_bom.drop_duplicates()

1. (Start) Find a product in production

In [194]:
product_production = df_results_production.loc[2].copy()

In [195]:
product_production['order_id'] = 42

In [196]:
product_production

location                         obninsk
product                  reinforcedrebar
bomnum           reinforcedrebarobninsk1
period                                 0
solutionvalue                    166.667
order_id                              42
Name: 2, dtype: object

2. Find the product's bom

In [197]:
df_product_bom = df_bom[(df_bom['bomnum'] == product_production['bomnum']) & (df_bom['period'] == product_production['period']) & (df_bom['input_output'] < 0)]
df_product_bom

,bomnum,location,product,input_output,period
17,reinforcedrebarobninsk1,obninsk,rebar,-1.050,0
18,reinforcedrebarobninsk1,obninsk,lime,-0.500,0


3. Get the first bom item

In [199]:
i = 0
order = df_product_bom.iloc[i].copy()
order

bomnum          reinforcedrebarobninsk1
location                        obninsk
product                           rebar
input_output                     -1.050
period                                0
Name: 17, dtype: object

4. Form an order

In [230]:
order['order_id'] = product_production['order_id']
order['loc_from'], order['loc_to'] = order['location'], order['location']
order['value'] = -order['input_output'] * product_production['solutionvalue']
order['residual'] = order['value']
order['leftover'] = order['value']
order['type'] = 'supply'

In [206]:
order

bomnum          reinforcedrebarobninsk1
location                        obninsk
product                           rebar
input_output                     -1.050
period                                0
order_id                             42
loc_from                        obninsk
loc_to                          obninsk
value                           175.000
residual                        175.000
leftover                        175.000
type                             supply
Name: 17, dtype: object

5. Recursive function call

In [207]:
df_production = df_results_production.copy()
df_production['type'] = 'production'
df_production['loc_from'], df_production['loc_to'] = df_results_production['location'], df_results_production['location']
df_production['leftover'] = df_results_production['solutionvalue']
df_production['value'] = df_results_production['solutionvalue']
compare_cols = ['value', 'leftover']

In [208]:
df_stock = df_results_stock.copy()
df_stock['type'] = 'stock'
df_stock['loc_from'], df_stock['loc_to'] = df_results_stock['location'], df_results_stock['location']
df_stock['leftover'] = df_results_stock['solutionvalue']
df_stock['value'] = df_results_stock['solutionvalue']

In [209]:
df_movement = df_results_movement.copy()
df_movement['type'] = 'movement'
df_movement['leftover'] = df_results_movement['solutionvalue']
df_movement['value'] = df_results_movement['solutionvalue']

In [210]:
# find a product at the same location and same period in production
df_production[(df_production['product'] == order['product']) &
                                      (df_production['period'] == order['period']) &
                                      (df_production['loc_to'] == order['loc_from']) &
                                      (df_production['leftover'] > 0) &
                                      # suppress selection from self leftovers
                                      (~df_production[compare_cols].eq(order[compare_cols]).all(axis=1) |
                                       (df_production['type'] != order['type']))]

,location,product,bomnum,period,solutionvalue,type,loc_from,loc_to,leftover,value
1,obninsk,rebar,rebarobninsk1,0,333.333,production,obninsk,obninsk,333.333,333.333


In [211]:
df_stock[(df_stock['product'] == order['product']) &
                                        (df_stock['period'] <= order['period']) &
                                        # movement from current period - 1
                                        (df_stock['period'] >= order['period'] - 1) &
                                        (df_stock['loc_to'] == order['loc_from']) &
                                        (df_stock['leftover'] > 0) &
                                        # suppress selection from self leftovers
                                        (~df_stock[compare_cols].eq(order[compare_cols]).all(axis=1) |
                                         (df_stock['type'] != order['type']))]

,location,product,period,solutionvalue,initialstock,value,type,loc_from,loc_to,leftover
11,obninsk,rebar,0,48.389,100.000,48.389,stock,obninsk,obninsk,48.389


In [212]:
df_movement[(df_movement['product'] == order['product']) &
                                              (df_movement['period'] <= order['period']) &
                                              # movement from current period - 1
                                              (df_movement['period'] >= order['period'] - 1) &
                                              (df_movement['loc_to'] == order['loc_from']) &
                                              (df_movement['leftover'] > 0) &
                                              # suppress selection from self leftovers
                                              (~df_movement[compare_cols].eq(order[compare_cols]).all(axis=1) |
                                               (df_movement['type'] != order['type']))]

,loc_from,loc_to,product,period,solutionvalue,transport_type,type,leftover,value
0,kemerovo,obninsk,rebar,0,100.000,auto,movement,100.000,100.000
9,omsk,obninsk,rebar,0,90.056,auto,movement,90.056,90.056


6. Get the second bom item

In [214]:
i = 1
order = df_product_bom.iloc[i].copy()
order

bomnum          reinforcedrebarobninsk1
location                        obninsk
product                            lime
input_output                     -0.500
period                                0
Name: 18, dtype: object

7. Form second order

In [215]:
order['order_id'] = product_production['order_id']

In [216]:
order['loc_from'], order['loc_to'] = order['location'], order['location']

In [217]:
order['value'] = -order['input_output'] * product_production['solutionvalue']

In [218]:
order['residual'] = order['value']

In [219]:
order['leftover'] = order['value']

In [220]:
order['type'] = 'supply'

In [221]:
order

bomnum          reinforcedrebarobninsk1
location                        obninsk
product                            lime
input_output                     -0.500
period                                0
order_id                             42
loc_from                        obninsk
loc_to                          obninsk
value                            83.333
residual                         83.333
leftover                         83.333
type                             supply
Name: 18, dtype: object

In [ ]:
8. Next interation of recursive calls

In [227]:
# find a product at the same location and same period in production
df_production[(df_production['product'] == order['product']) &
                                      (df_production['period'] == order['period']) &
                                      (df_production['loc_to'] == order['loc_from']) &
                                      (df_production['leftover'] > 0) &
                                      # suppress selection from self leftovers
                                      (~df_production[compare_cols].eq(order[compare_cols]).all(axis=1) |
                                       (df_production['type'] != order['type']))]

,location,product,bomnum,period,solutionvalue,type,loc_from,loc_to,leftover,value


In [228]:
df_stock[(df_stock['product'] == order['product']) &
                                        (df_stock['period'] <= order['period']) &
                                        # movement from current period - 1
                                        (df_stock['period'] >= order['period'] - 1) &
                                        (df_stock['loc_to'] == order['loc_from']) &
                                        (df_stock['leftover'] > 0) &
                                        # suppress selection from self leftovers
                                        (~df_stock[compare_cols].eq(order[compare_cols]).all(axis=1) |
                                         (df_stock['type'] != order['type']))]

,location,product,period,solutionvalue,initialstock,value,type,loc_from,loc_to,leftover
3,obninsk,lime,0,416.667,500.000,416.667,stock,obninsk,obninsk,416.667


In [229]:
df_movement[(df_movement['product'] == order['product']) &
                                              (df_movement['period'] <= order['period']) &
                                              # movement from current period - 1
                                              (df_movement['period'] >= order['period'] - 1) &
                                              (df_movement['loc_to'] == order['loc_from']) &
                                              (df_movement['leftover'] > 0) &
                                              # suppress selection from self leftovers
                                              (~df_movement[compare_cols].eq(order[compare_cols]).all(axis=1) |
                                               (df_movement['type'] != order['type']))]

,loc_from,loc_to,product,period,solutionvalue,transport_type,type,leftover,value


In [224]:
df_procurement = df_results_procurement.copy()
df_procurement['type'] = 'procurement'
df_procurement['loc_from'], df_procurement['loc_to'] = df_results_procurement['location'], df_results_procurement['location']
df_procurement['leftover'] = df_results_procurement['solutionvalue']
df_procurement['value'] = df_results_procurement['solutionvalue']

In [225]:
df_procurement[(df_procurement['product'] == order['product']) &
                                              (df_procurement['period'] <= order['period']) &
                                              # movement from current period - 1
                                              (df_procurement['period'] >= order['period'] - 1) &
                                              (df_procurement['loc_to'] == order['loc_from']) &
                                              (df_procurement['leftover'] > 0) &
                                              # suppress selection from self leftovers
                                              (~df_procurement[compare_cols].eq(order[compare_cols]).all(axis=1) |
                                               (df_procurement['type'] != order['type']))]

,location,product,period,solutionvalue,supplier,type,loc_from,loc_to,leftover,value


# Close connection

In [83]:
conn.commit()

In [84]:
# Close the cursor and the database connection
cursor.close()
conn.close()